In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df_game = pd.read_csv('dados_case_sr.csv', sep=';', error_bad_lines=False)
df_game.head()

,Unnamed: 0,device_id,event_date,creation_date,app,platform,partner,country,iap_purchases,iap_revenue,ads_revenue,ad_impressions
0,1,484c0f6c716b62fbdd651b48448632c6,2022-01-05,2020-12-31,Football Simulator,android,Organic,Canada,1,"4,18164547135849","0,00268264452",6
1,2,54a09dd2892b768a6ae566e0f4ee61e1,2022-05-29,2022-05-01,Football Simulator,android,Paid,Pakistan,1,"2,35950024395138","0,0652680554058738",3
2,3,be59fd534f61e87e6c617fa50edde27e,2022-04-02,2022-03-30,Football Simulator,ios,Organic,United States,1,"2,22307474997593","0,03285518396",28
3,4,af4830573c2ac1d4efc538d16d645b31,2022-03-25,2022-03-25,Football Simulator,android,Organic,Argentina,1,"3,93345382196093","0,00025961076",7
4,5,13c70063c611c94b1f5051e4eba6f5c0,2022-03-24,2022-03-12,Football Simulator,android,Paid,United States,1,"3,23837452336857","0,00066344972",4


## Verificando a necessidade de ajustes na base

#### Analisando o shape da base:

In [3]:
print(df_game.shape[0], 'X' , df_game.shape[1])

9067690 X 12


In [4]:
df_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9067690 entries, 0 to 9067689
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Unnamed: 0      int64 
 1   device_id       object
 2   event_date      object
 3   creation_date   object
 4   app             object
 5   platform        object
 6   partner         object
 7   country         object
 8   iap_purchases   int64 
 9   iap_revenue     object
 10  ads_revenue     object
 11  ad_impressions  int64 
dtypes: int64(3), object(9)
memory usage: 830.2+ MB


#### Ajustes necessarios:
* deletar primeira coluna, ela na verdade é somente um index
* **event_date** e **creation_date** tem que ser ajustada para datetime , autalmente está sendo tratada como object
* **iap_revenue** e **ads_revenue** estão como object , tem que deixar como float. Será necessario mudar a pontuação usada
* deixar o resto como string

### Verificando a existencia de duplicadas

In [5]:
df_game.isna().sum()

Unnamed: 0            0
device_id             0
event_date            0
creation_date         0
app                   0
platform              0
partner               0
country           20285
iap_purchases         0
iap_revenue           0
ads_revenue           0
ad_impressions        0
dtype: int64

In [6]:
dados_nulos = df_game.isna().sum()
dados_totais = df_game.shape[0]
percentual_de_zeros = (dados_nulos / dados_totais)*100
print(percentual_de_zeros)

Unnamed: 0        0.000000
device_id         0.000000
event_date        0.000000
creation_date     0.000000
app               0.000000
platform          0.000000
partner           0.000000
country           0.223706
iap_purchases     0.000000
iap_revenue       0.000000
ads_revenue       0.000000
ad_impressions    0.000000
dtype: float64


Apenas a feature **Country** possui dados faltosos. Eles representam apenas 0,22% do total, logo não deverá ter tanto impacto deleta-los

#### Realizando ajustes 

In [7]:
# Deletar coluna de axis
df_game = df_game.drop('Unnamed: 0' , axis=1)

In [8]:
df_game.head()

,device_id,event_date,creation_date,app,platform,partner,country,iap_purchases,iap_revenue,ads_revenue,ad_impressions
0,484c0f6c716b62fbdd651b48448632c6,2022-01-05,2020-12-31,Football Simulator,android,Organic,Canada,1,"4,18164547135849","0,00268264452",6
1,54a09dd2892b768a6ae566e0f4ee61e1,2022-05-29,2022-05-01,Football Simulator,android,Paid,Pakistan,1,"2,35950024395138","0,0652680554058738",3
2,be59fd534f61e87e6c617fa50edde27e,2022-04-02,2022-03-30,Football Simulator,ios,Organic,United States,1,"2,22307474997593","0,03285518396",28
3,af4830573c2ac1d4efc538d16d645b31,2022-03-25,2022-03-25,Football Simulator,android,Organic,Argentina,1,"3,93345382196093","0,00025961076",7
4,13c70063c611c94b1f5051e4eba6f5c0,2022-03-24,2022-03-12,Football Simulator,android,Paid,United States,1,"3,23837452336857","0,00066344972",4


In [9]:
# Arrumando colunas de data
df_game['event_date'] = pd.to_datetime(df_game['event_date'])
df_game['creation_date'] = pd.to_datetime(df_game['creation_date'])

In [10]:
# Arrumando as valores iap_revenue e ads_revenue para formato de valor
df_game['iap_revenue'] = df_game['iap_revenue'].str.replace(',' , '.').astype(float)
df_game['ads_revenue'] = df_game['ads_revenue'].str.replace (',' , '.').astype(float)


In [11]:
'''lista_edicoes=['app' , 'platform' , 'partner' , 'country']
for n in lista_edicoes:
    df_game[n] = str(df_game[n])'''

"lista_edicoes=['app' , 'platform' , 'partner' , 'country']\nfor n in lista_edicoes:\n    df_game[n] = str(df_game[n])"

In [12]:
# deletando linhas com country vazio
df_game.dropna(inplace = True)

In [13]:
df_game.head()

,device_id,event_date,creation_date,app,platform,partner,country,iap_purchases,iap_revenue,ads_revenue,ad_impressions
0,484c0f6c716b62fbdd651b48448632c6,2022-01-05,2020-12-31,Football Simulator,android,Organic,Canada,1,4.181645,0.002683,6
1,54a09dd2892b768a6ae566e0f4ee61e1,2022-05-29,2022-05-01,Football Simulator,android,Paid,Pakistan,1,2.359500,0.065268,3
2,be59fd534f61e87e6c617fa50edde27e,2022-04-02,2022-03-30,Football Simulator,ios,Organic,United States,1,2.223075,0.032855,28
3,af4830573c2ac1d4efc538d16d645b31,2022-03-25,2022-03-25,Football Simulator,android,Organic,Argentina,1,3.933454,0.000260,7
4,13c70063c611c94b1f5051e4eba6f5c0,2022-03-24,2022-03-12,Football Simulator,android,Paid,United States,1,3.238375,0.000663,4


In [14]:
df_game_2022 = df_game.loc[df_game['creation_date']>= '2022-01-01']

## Analisando os dados

## 1) Plataforma

#### Número de jogadores

In [15]:
df_plat = df_game_2022['platform'].value_counts() / df_game_2022.shape[0]
df_plat

android    0.725771
ios        0.274199
amazon     0.000030
Name: platform, dtype: float64

In [16]:
df_plat2 = df_game_2022.groupby('platform').device_id.count()
df_plat2

platform
amazon         224
android    5347430
ios        2020278
Name: device_id, dtype: int64

Conclusão : Android é a sistema operacional com mais dowloads, cerca de 72,7% (5 Mi devices) , seguido por IOS 27,2% (1.9 Mi devices). Amazon quase não é relevante (213 devices)

#### Número de compras

In [17]:
df_plat_compras = df_game_2022.groupby('platform').iap_purchases.sum()
df_plat_compras

platform
amazon         224
android    5419986
ios        2055518
Name: iap_purchases, dtype: int64

Conclusão : Android é a sistema operacional onde os jogadores mais realizaram compras, cerca de 5.3 Mi de compras.

#### Plataforma com maior revenue

In [18]:
df_game_2022['revenue_total'] = df_game_2022['iap_revenue'] + df_game_2022['ads_revenue']

<ipython-input-18-eb6f24340110>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game_2022['revenue_total'] = df_game_2022['iap_revenue'] + df_game_2022['ads_revenue']


In [19]:
round(df_game_2022.loc[df_game_2022['platform'] == 'amazon'].revenue_total.sum() ,2)

668.85

In [20]:
round(df_game_2022.loc[df_game_2022['platform'] == 'android'].revenue_total.sum(),2)

15906425.92

In [21]:
round(df_game_2022.loc[df_game_2022['platform'] == 'ios'].revenue_total.sum(),2)

6043605.83

Conclusão : Android é a sistema operacional que trouxe mais revenue , cerca de BRL 15 Mi , IOS trouxe BRL 5.8 Mi

#### Retorno médio do jogador por plataforma

In [22]:
round(df_game_2022.loc[df_game_2022['platform'] == 'android'].revenue_total.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'android'].device_id.count()

2.9745926398288525

In [23]:
round(df_game_2022.loc[df_game_2022['platform'] == 'ios'].revenue_total.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'ios'].device_id.count()

2.9914723765739173

Conclusão : o ganho médio com IOS é levente maior que com Android

#### Ticket médio do jogador

In [24]:
round(df_game_2022.loc[df_game_2022['platform'] == 'android'].iap_purchases.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'android'].device_id.count()

1.01356838705696

In [25]:
round(df_game_2022.loc[df_game_2022['platform'] == 'ios'].iap_purchases.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'ios'].device_id.count()

1.0174431439633556

In [26]:
round(df_game_2022.loc[df_game_2022['platform'] == 'amazon'].iap_purchases.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'amazon'].device_id.count()

1.0

Conclusão : o ticket médio dé praticamente o mesmo independente da plataforma 

#### Impressoes de anuncio por jogador em cada plataforma

In [27]:
round(df_game_2022.loc[df_game_2022['platform'] == 'android'].ad_impressions.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'android'].device_id.count()

13.333372292858439

In [28]:
round(df_game_2022.loc[df_game_2022['platform'] == 'ios'].ad_impressions.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'ios'].device_id.count()

10.64888545041821

In [29]:
round(df_game_2022.loc[df_game_2022['platform'] == 'amazon'].ad_impressions.sum(),2) / df_game_2022.loc[df_game_2022['platform'] == 'amazon'].device_id.count()

3.0

Conclusão : Jogadores que utilziam como plataforma o android são mais impactados por anuncios

In [30]:
# quem fica mais tempo jogando ?

## 2) Country 

#### País com maior número de dowloads

In [31]:
df_pais_jogadores = df_game_2022.groupby('country').device_id.count()
df_pais_jogadores = pd.DataFrame(df_pais_jogadores).reset_index() 
df_pais_jogadores = df_pais_jogadores.sort_values("device_id" , ascending=False)
df_pais_jogadores

,country,device_id
221,United States,1564465
29,Brazil,609061
96,India,483780
78,Germany,455712
220,United Kingdom,358499
...,...,...
65,Eritrea,4
178,Saint-Barthélemy,3
190,Solomon Islands,2
110,Kiribati,2


Conclusão: EUA lideram em número de downloads, estando a mais que o dobro do segundo colocado, o Brasil

In [32]:
df_pais_valor = df_game_2022.groupby('country').revenue_total.sum()
df_pais_valor = pd.DataFrame(df_pais_valor).reset_index() 
df_pais_valor = df_pais_valor.sort_values("revenue_total" , ascending=False)
df_pais_valor

,country,revenue_total
221,United States,4.751085e+06
29,Brazil,1.787848e+06
96,India,1.419299e+06
78,Germany,1.365076e+06
220,United Kingdom,1.070884e+06
...,...,...
65,Eritrea,8.427658e+00
178,Saint-Barthélemy,7.583479e+00
110,Kiribati,5.324944e+00
190,Solomon Islands,4.197952e+00


Conclusão: EUA lideram em revenue

## 3)Partner

#### Qual partner trouxe mais jogadores ?

In [33]:
df_partner = df_game_2022['partner'].value_counts() / df_game_2022.shape[0]
df_partner

Paid       0.571598
Organic    0.428402
Name: partner, dtype: float64

Conclusão: Há mais jogadores de origem paga

#### Qual partner traz mais revenue ?

In [34]:
round(df_game_2022.loc[df_game_2022['partner'] == 'Paid'].revenue_total.sum(),2)

12553804.26

In [35]:
round(df_game_2022.loc[df_game_2022['partner'] == 'Organic'].revenue_total.sum(),2)

9396896.34

Conclusão : Jogadores que vieram via user acquisitio gastam mais

#### Qual retorno médio por jogador trazido por cada partner?

In [36]:
round(df_game_2022.loc[df_game_2022['partner'] == 'Paid'].revenue_total.sum(),2) / df_game_2022.loc[df_game_2022['partner'] == 'Paid'].device_id.count()

2.980844147194356

In [37]:
round(df_game_2022.loc[df_game_2022['partner'] == 'Organic'].revenue_total.sum(),2) / df_game_2022.loc[df_game_2022['partner'] == 'Organic'].device_id.count()

2.9770562142971873

Conclusão : O retorno médio por jogador é praticamente igual, idependente da origem dele

In [38]:
# com dummies para categoricas
df_game = pd.get_dummies(df_game , columns=['platform' , 'partner' , 'country'])

In [39]:
'''
# deixar no formato mes
df_game['event_date'] = pd.to_datetime(df_game['event_date'],format="%Y-%m-%D")
df_game['event_date'] = [i.strftime('%Y-%m') for i in df_game['event_date']]
df_game['creation_date'] = pd.to_datetime(df_game['creation_date'],format="%Y-%m-%D")
df_game['creation_date'] = [i.strftime('%Y-%m') for i in df_game['creation_date']]
'''

In [41]:
#df_game_mes = df_game

In [42]:
#df_game_mes.to_csv('game_mes.csv', index=False)

In [44]:
#df_game_mes22 = df_game_mes.loc[df_game_mes['creation_date']>= '2022-01']

In [40]:
# agrupar por data de primeiro login
#df_dt_login = df_game_.groupby('creation_date').sum()
#df_dt_login

In [45]:
#df_game_mes22.to_csv('game_mes22.csv', index=False)